# 133. Clone Graph

[link](https://leetcode.com/problems/clone-graph/description/)

## Understand the task

Understand the given data structure:

In [2]:
# Definition for a Node.
class Node(object):
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []


- Two instance variables: integer value and a vector with pointers to other nodes connected to this node
- Three overloaded constructors: no value and no neighbours, a value with no neighbours, and a value with a vector of neighbours

## Brute-force algorithm

- We start at the first node with `val == 1` and create a copy
- We iterate over the neighbours and copy them
- Repeat the previous step for each neighbour of the neighbour, etc.
- Since we need the code to branch out, it is a use case for recursion
- Base cases:
  - a node with no neighbours - no need to branch out altogether
  - a node with no *new* neighbours, i.e. the nodes we haven't copied before
  - from looking at the test cases, we can see that an empty graph (with no nodes) is also a possibility 

  This means we need to keep track of the nodes we have already seen. How are we going to do that?
  Read the task again:
  
  > For simplicity, each node's value is the same as the node's index (1-indexed). For example, the first node with val == 1, the second node with val == 2, and so on.

  This means that each node has a unique value. We can use that to keep track of the nodes previously encountered. We can store the values in a hash table or a list, which will enable us to search for the desired values in O(1). Node values correspond to their indices, so, for simplicity, we can use a list of numbers where each index also corresponds to the value of each node (with the 0th index being empty).

  Length of the list could potentially be an issue because it takes up a fixed amount of memory; however, the task states that the length is always in the range `[0, 100]`. We can see the length of the graph by looking at the adjacency list in each test case, but our algorithm won't have access to that list and won't know the size of the graph in advance. Hence, we will have to use a list with the maximum length of 100. 

From here, we start implementing the brute-force solution:

In [ ]:
class Solution0(object):
  def cloneGraph(self, node):
    """
    :type node: Node
    :rtype: Node
    """

    # Create the list to keep track of visited nodes
    visitedNodes = [0] * 100

    # Copy this node and save its value
    copiedGraph = Node(node.val, node.neighbors)
    if visitedNodes[node.val] == 0:
      visitedNodes[node.val] == node.val

    # Iterate over neighbours
    for i in range(1, len(node.neighbors) + 1):
      n = node.neighbors[i]

      if visitedNodes[n.val] == 0:
        visitedNodes[n.val] = n.val

        # Call the function on the neighbour and attach the returned node to the current one
        copiedGraph.neighbors[i] = someRecursiveFunction(n)


At this point, we want to extract the piece of code that iterates over the neighbours into a separate function that we can call recursively, and add base cases to it.

We can also take advantage of the fact that our solution is a class and turn `visitedNodes` into an instance variable that all of our methods will have access to.

In [ ]:
class Solution1(object):
    def __init__(self):
        # Create the list to keep track of visited nodes
        # (Recall: node values start with 1, so total length will be 101)
        self.visitedNodes = [None] * 101


    def copyNodeAndNeighbours(self, currentNode):
        print("current node value", currentNode.val)
        for nei in currentNode.neighbors:
            print(nei.val)

        # Base case 1: a node with no neighbours
        if len(currentNode.neighbors) == 0:
            return Node(currentNode.val)

        else:
            copiedNode = Node(currentNode.val, [None] * len(currentNode.neighbors))
    
            # Iterate over neighbours
            for i in range(0, len(currentNode.neighbors)):
                if self.visitedNodes[currentNode.neighbors[i].val] == None:
                    self.visitedNodes[currentNode.neighbors[i].val] = currentNode.neighbors[i].val

                    # Call the function on the neighbour
                    # Attach the returned node to the current one
                    copiedNode.neighbors[i] = self.copyNodeAndNeighbours(n)

                else:
                    # Base case 2: a node with no new neighbours
                    copiedNode.neighbors[i] = n

            return copiedNode


    def cloneGraph(self, node):
        """
        :type node: Node
        :rtype: Node
        """
        # Base case 3: empty graph
        if node == None:
            return None
        if node.val == 0:
            return Node()

        # Save this node's value
        self.visitedNodes[node.val] == node.val

        # Start the recursive function
        copiedGraph = self.copyNodeAndNeighbours(Node(node.val, node.neighbors))

        return copiedGraph


This code works for the base cases, but the recursive test case returns a wrong answer:

> ```Node with value 2 was not copied but a reference to the original one.```

This means that we are returning the original node somewhere in the code.

What if, instead of keeping track of the vertices that we have visited, we keep track of nodes?

We know from the task that there are no parallel vertices or loops, so each pair of connected vertices has a unique edge between them.

How can we keep track of the edges? Each edge can be represented by a tuple with the names of its vertices: `(1, 2)`. Or we can have a dictionary that keeps track of the connections:

```
{
  1: [2, 4],
  2: [1, 3],
  ...
}
```

which is similar to the adjacency list that represents the test cases.

Try the new approach:

In [ ]:
class Solution2(object):
    def __init__(self):
        # Create the list to keep track of visited edges
        self.visitedEdges = dict({1: []})


    def copyNodeAndNeighbours(self, currentNode):
        print("current node value", currentNode.val)
        for nei in currentNode.neighbors:
            print(nei.val)

        # Base case 1: a node with no neighbours
        if len(currentNode.neighbors) == 0:
            return Node(currentNode.val)

        else:
            copiedNode = Node(currentNode.val, [None] * len(currentNode.neighbors))
    
            # Iterate over neighbours
            for i in range(0, len(currentNode.neighbors)):
                currentNeighbor = currentNode.neighbors[i]
                currentEdge = currentNeighbor.val

                if (
                    (not self.visitedEdges[copiedNode.val] or not currentEdge in self.visitedEdges[copiedNode.val])
                    or (not self.visitedEdges[currentEdge] or not copiedNode.val in self.visitedEdges[currentEdge])
                ):
                    if self.visitedEdges[copiedNode.val]:
                        self.visitedEdges[copiedNode.val].append(currentEdge)
                    else:
                        self.visitedEdges[copiedNode.val] = [currentEdge]
                    if currentEdge in self.visitedEdges:
                        self.visitedEdges[currentEdge].append(copiedNode.val)
                    else:
                        self.visitedEdges[currentEdge] = [copiedNode.val]

                    # Proceed down that edge by calling the function on the neighbour
                    # Attach the returned node to the current one
                    copiedNode.neighbors[i] = self.copyNodeAndNeighbours(Node(
                        currentNeighbor.val,
                        currentNeighbor.neighbors
                    ))

                else:
                    # We have already traversed this edge
                    copiedNode.neighbors[i] = currentNeighbor

            return copiedNode


    def cloneGraph(self, node):
        """
        :type node: Node
        :rtype: Node
        """
        # Base case 3: empty graph
        if node == None:
            return None
        if node.val == 0:
            return Node()

        # Start the recursive function
        copiedGraph = self.copyNodeAndNeighbours(node)

        return copiedGraph

This approach results in the same problem. Because the logic of tracking edges is more convoluted, we will go back to tracking vertices and inspect the problem again.

The problem likely comes from the fact that we are iterating over the pointers to the neighbors from the original graph before we create copies of them. So for the vertices that we have already seen, we are copying the originals.

We need a way of referencing the nodes we have already copied. To do that, instead of keeing track of the values we have already seen, we can save pointers to the copied nodes that contain those values. Then we can use those pointers to link together the nodes that have been already seen. Let's try that.

In [ ]:
class Solution3(object):
    def __init__(self):
        # Create the list to keep track of visited nodes
        # (Recall: node values start with 1, so total length will be 101)
        self.visitedNodes = [None] * 101


    def copyNodeAndNeighbours(self, currentNode):
        print("current node value", currentNode.val)
        for nei in currentNode.neighbors:
            print(nei.val)

        # Base case 1: a node with no neighbours
        if len(currentNode.neighbors) == 0:
            return Node(currentNode.val)

        else:
            copiedNode = Node(currentNode.val, [None] * len(currentNode.neighbors))
            self.visitedNodes[copiedNode.val] = copiedNode

            for i in range(0, len(copiedNode.neighbors)):
                n = currentNode.neighbors[i]
                
                if self.visitedNodes[n.val] == None:
                    # We haven't visited this neighbour yet
                    copiedNode.neighbors[i] = self.copyNodeAndNeighbours(n)

                else:
                    copiedNode.neighbors[i] = self.visitedNodes[n.val]
            
            return copiedNode


    def cloneGraph(self, node):
        """
        :type node: Node
        :rtype: Node
        """
        # Base case 3: empty graph
        if node == None:
            return None
        if node.val == 0:
            return Node()

        # Start the recursive function
        copiedGraph = self.copyNodeAndNeighbours(Node(node.val, node.neighbors))

        return copiedGraph


We have found a working solution. But it has suboptimal performance:

> **Runtime**
>
> 42ms
>
> Beats 5.62% of users with Python
>
> **Memory**
>
> 11.92MB
>
> Beats 56.71% of users with Python
>

## Optimizing for space

The parts of our algorithm that use the most memory are:

* Recursion - uses up stack memory
* List of fixed length 101 - uses the same amount of memory regardless of the size of each given graph

Due to the nature of graphs, we can't avoid the branching code, so we will not try to convert the recursive function into a tail-recursive one to save space.

However, we can reduce the size of our list of visited vertices by using a hash table (Python dictionary) instead.

In [ ]:
class Solution4(object):
    def __init__(self):
        # Create the list to keep track of visited nodes
        # (Recall: node values start with 1, so total length will be 101)
        self.visitedNodes = dict({})


    def copyNodeAndNeighbours(self, currentNode):
        print("current node value", currentNode.val)
        for nei in currentNode.neighbors:
            print(nei.val)

        # Base case 1: a node with no neighbours
        if len(currentNode.neighbors) == 0:
            return Node(currentNode.val)

        else:
            copiedNode = Node(currentNode.val, [None] * len(currentNode.neighbors))
            self.visitedNodes[copiedNode.val] = copiedNode

            for i in range(0, len(copiedNode.neighbors)):
                n = currentNode.neighbors[i]
                
                if not n.val in self.visitedNodes:
                    # We haven't visited this neighbour yet
                    copiedNode.neighbors[i] = self.copyNodeAndNeighbours(n)

                else:
                    copiedNode.neighbors[i] = self.visitedNodes[n.val]
            
            return copiedNode


    def cloneGraph(self, node):
        """
        :type node: Node
        :rtype: Node
        """
        # Base case 3: empty graph
        if node == None:
            return None
        if node.val == 0:
            return Node()

        # Start the recursive function
        copiedGraph = self.copyNodeAndNeighbours(Node(node.val, node.neighbors))

        return copiedGraph


The memory use is much improved, and the rumtime is also better now:

> **Runtime**
>
> 35ms
>
> Beats 30.32% of users with Python
>
> **Memory**
>
> 11.74MB
>
> Beats 97.68% of users with Python

## Optimizing for time

We can at this point combine the base cases of `cloneGraph()` and `copyNodeAndNeighbours()` and turn them into one method.

We should also remove the print statement with the loop, which we used for debugging and then forgot about :facepalm:

In [ ]:
class Solution5(object):
    def __init__(self):
        # Create the list to keep track of visited nodes
        # (Recall: node values start with 1, so total length will be 101)
        self.visitedNodes = dict({})

    def cloneGraph(self, node):
        """
        :type node: Node
        :rtype: Node
        """
        # Base case 3: empty graph
        if node == None:
            return None
        if node.val == 0:
            return Node()

        # Base case 1: a node with no neighbours
        if len(node.neighbors) == 0:
            return Node(node.val)

        else:
            copiedNode = Node(node.val, [None] * len(node.neighbors))
            self.visitedNodes[copiedNode.val] = copiedNode

            for i in range(0, len(copiedNode.neighbors)):
                n = node.neighbors[i]
                
                if not n.val in self.visitedNodes:
                    # We haven't visited this neighbour yet
                    copiedNode.neighbors[i] = self.cloneGraph(n)

                else:
                    copiedNode.neighbors[i] = self.visitedNodes[n.val]
            
            return copiedNode


As expected, moving more logic into the recursive method slightly increased memory use; however, the runtime improved significantly:

> **Runtime**
>
> 25ms
>
> Beats 91.22% of users with Python
>
> **Memory**
>
> 11.90MB
>
> Beats 84.49% of users with Python>